In [1]:
# time: 2023.07.26
# author: bio大恐龙

# define a function to get website infomation with html format
import requests
from bs4 import BeautifulSoup

def get_url_info(url):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'}
    try:
        info = requests.get(url, headers=headers).text.encode('gbk', 'ignore').decode('gbk')
        soup = BeautifulSoup(info, 'html.parser')
        return soup
    except:
        print('Sorry! The film information is not got')

# define a function to download film poster
def download_image(url, save_path):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'}
    try:
        image = requests.get(url, headers=headers).content
        with open(save_path, 'wb') as f:
            f.write(image)
    except:
        print('Sorry! failure to download the image')

In [2]:
import pandas as pd
import time

# construct a dataframe to store movies shenteng involved in information
shenteng_movies_df = pd.DataFrame(columns=['Film_Name', 'URL', 'Year'])

'''
the urls of website were constructed as following url with difference in "start" and total pages are 13
'https://movie.douban.com/celebrity/1325700/movies?start=10&format=pic&sortby=time&'
'https://movie.douban.com/celebrity/1325700/movies?start=20&format=pic&sortby=time&'
'''
df_index = 0
website_list = list(range(1,14))

for i in website_list:
    movie_info = get_url_info(f'https://movie.douban.com/celebrity/1325700/movies?start={i}0&format=pic&sortby=time&')
    interest_info = movie_info.find_all('h6')
    #print(interest_info[0].find('span'))
    #break
    for k in range(len(interest_info)):
        movie_year = interest_info[k].find('span').text.strip('()')
        movie_url = interest_info[k].find('a')['href']
        movie_name = interest_info[k].find('a').text
        shenteng_movies_df.loc[df_index] = [movie_name, movie_url, movie_year]
        df_index += 1
    time.sleep(10)
    

In [7]:
# shenteng_movies_df.to_csv('/mnt/c/Users/ouyangkang/Desktop/basic_info.csv', index=None, encoding='gbk')

In [3]:
import json
import os
from bs4.element import Tag
import re

# create a directory to store the posters of film
dir_path = '/mnt/c/Users/ouyangkang/Desktop/film_poster/'
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
    
# construct a dataframe to store new infomation of films
films_detail_df = pd.DataFrame(columns=['Film_name', 'URL', 'Year', 'Director', 'Actors', 'Genre', 'Rating_count', 'Rating_value', 'IMDb', 'Description', 'Interesting_count', 'Watched_count'])
# index
initial_number = 0

for single_movie_url in shenteng_movies_df['URL'].tolist():
    time.sleep(4)
    movie_info = get_url_info(single_movie_url)

    # screen non-film infomation and not yet shown
    if isinstance(movie_info.find('div', class_="rating_sum"), Tag):
        if "暂无" not in movie_info.find('div', class_="rating_sum").text and "尚未" not in movie_info.find('div', class_="rating_sum").text: 
            # construct directory data foramt
            json_data = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', movie_info.find('script', type="application/ld+json").string.strip())
            json_data = json.loads(json_data)
            
            if json_data['@type'] == 'Movie' and json_data['aggregateRating']['ratingValue'] != "" and json_data['description'] != "" and "真人秀" not in json_data['genre'] and "脱口秀" not in json_data['genre'] and '歌舞' not in json_data['genre']:
                # name
                name = shenteng_movies_df[shenteng_movies_df["URL"] == single_movie_url]['Film_Name'].tolist()[0]
                # url
                url = single_movie_url
                # year
                year = shenteng_movies_df[shenteng_movies_df["URL"] == single_movie_url]['Year'].tolist()[0]
                # director
                director = json_data['director'][0]['name'].split()[0]
                # actors
                actors = str([i['name'].split()[0] for i in json_data['actor']]).strip('[]') # only chinese name
                # genre
                genre = str(json_data['genre']).strip('[]')
                # rating count
                rating_count = json_data['aggregateRating']['ratingCount']
                # rating value
                rating_value = json_data['aggregateRating']['ratingValue']
                # IMDb
                if isinstance(movie_info.find('span', class_='pl', text="IMDb:"), Tag):
                    imdb = movie_info.find('span', class_='pl', text="IMDb:").next_sibling.strip()
                else:
                    imdb = None
                # description
                description = json_data['description']
                # how many people are interested in the film and had watched
                if isinstance(movie_info.find("div", class_="subject-others-interests-ft"), Tag):
                    tem_info = movie_info.find("div", class_="subject-others-interests-ft").find_all('a')
                    interest_count = tem_info[0].text.split('人')[0]
                    watched_count = tem_info[1].text.split('人')[0]
                else:
                    interest_count = None
                    watched_count = None
                # poster url
                image_url = json_data['image']
                
                films_detail_df.loc[initial_number] = [name, url, year, director, actors, genre, rating_count, rating_value, imdb, description, interest_count, watched_count]
                initial_number += 1

                time.sleep(8)
                save_path = dir_path + name + '.jpg'
                download_image(image_url, save_path)
                time.sleep(8)

In [37]:
films_detail_df.head()
# films_detail_df.to_csv('/mnt/c/Users/ouyangkang/Desktop/films_info.csv', index=None, encoding='gbk')